# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform

In [35]:
data = pd.read_csv('../data/customer_product_sales.csv').set_index('CustomerID')
data

,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
CustomerID,,,,,,
61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
...,...,...,...,...,...,...
94547,Gerald,Donovan,687884,211,Wine - Prosecco Valdobiaddene,24
57654,Lashonda,Roy,4373415,211,Wine - Prosecco Valdobiaddene,15
36751,Glen,Frederick,2405597,211,Wine - Prosecco Valdobiaddene,10


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [7]:
df1 = data.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index()
df1

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [60]:
pivot_table = pd.pivot_table(df1, values='Quantity', index='ProductName', columns='CustomerID', fill_value=0)
pivot_table = pivot_table.fillna(0)
pivot_table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [64]:
customer_similarity = pdist(pivot_table.T, metric='euclidean')
customer_similarity_matrix = pd.DataFrame(squareform(customer_similarity),\
                                          index=pivot_table.columns, columns=pivot_table.columns)
customer_similarity_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,220.501701,220.202180,219.136943,219.952268,219.733930,219.599636,219.538152,219.924987,219.827205,220.070443,...,283.945417,283.945417,302.076149,272.717803,278.388218,0.000000,273.861279,291.547595,306.186218,307.205143
98069,217.188858,215.728997,216.612557,217.437347,217.446545,217.425849,216.903204,217.294731,217.080630,216.751009,...,283.945417,283.945417,295.803989,283.945417,285.043856,273.861279,0.000000,287.228132,297.909382,294.745653
98159,228.628520,228.010965,228.081126,228.098663,227.997807,228.197283,228.028507,228.181945,227.868383,228.103047,...,283.945417,279.508497,300.000000,290.473751,300.000000,291.547595,287.228132,0.000000,304.138127,305.163890


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [66]:
customer_id = 33

similar_customers = customer_similarity_matrix[customer_id].sort_values(ascending=True)[1:6]
similar_customers

CustomerID
3535    10.488088
264     10.488088
3317    10.488088
2503    10.630146
3305    10.677078
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [68]:
similar_customer_ids = similar_customers.index.tolist()

selected_records = df1[df1['CustomerID'].isin(similar_customer_ids)]
selected_records

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [69]:
aggregated_records = selected_records.groupby('ProductName')['Quantity'].sum().reset_index()
ranked_records = aggregated_records.sort_values(by='Quantity', ascending=False)
ranked_records

,ProductName,Quantity
33,Butter - Unsalted,3
203,Wine - Ej Gallo Sierra Valley,3
186,Towels - Paper / Kraft,3
168,Soup - Campbells Bean Medley,3
199,Wine - Blue Nun Qualitatswein,3
...,...,...
85,Hinge W Undercut,1
86,Ice Cream Bar - Hageen Daz To,1
87,Jagermeister,1
88,Jolt Cola - Electric Blue,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [71]:
customer_id = 33

products_not_purchased = ranked_records[~ranked_records['ProductName']\
                                        .isin(pivot_table.index[pivot_table[customer_id] > 0])]

merged_df = products_not_purchased.merge(pivot_table, on='ProductName')

recommendations = merged_df[merged_df[customer_id] == 0]

top_5_recommendations = recommendations.head(5)
top_5_recommendations

,ProductName,Quantity,33,200,264,356,412,464,477,639,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Butter - Unsalted,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
1,Wine - Ej Gallo Sierra Valley,3,0,0,0,1,0,0,0,0,...,0,0,25,50,0,25,0,0,0,0
2,Soup - Campbells Bean Medley,3,0,0,1,0,0,0,2,0,...,0,0,0,25,50,25,0,0,0,0
3,Wine - Blue Nun Qualitatswein,3,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
4,Chicken - Soup Base,2,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [76]:
customer_ids = df1['CustomerID'].unique()
recommendations_dict = {}

for customer_id in customer_ids:
    similar_customers = customer_similarity_matrix[customer_id].sort_values(ascending=True)[1:6]

    selected_records = df1[df1['CustomerID'].isin(similar_customers.index.tolist())]

    aggregated_records = selected_records.groupby('ProductName')['Quantity'].sum().reset_index()
    ranked_records = aggregated_records.sort_values(by='Quantity', ascending=False)

    products_not_purchased = ranked_records[~ranked_records['ProductName'].isin(pivot_table.index[pivot_table[customer_id] > 0])]
    merged_df = products_not_purchased.merge(pivot_table, on='ProductName')
    recommendations = merged_df[merged_df[customer_id] == 0]
    top_5_recommendations = recommendations.head(5)

    recommendations_dict[customer_id] = top_5_recommendations['ProductName'].tolist()

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [80]:
recommendations_df = pd.DataFrame.from_dict(recommendations_dict, orient='index').reset_index()
recommendations_df.columns = ['CustomerID', 'Recommendation1', 'Recommendation2', \
                              'Recommendation3', 'Recommendation4', 'Recommendation5']
recommendations_df

,CustomerID,Recommendation1,Recommendation2,Recommendation3,Recommendation4,Recommendation5
0,33,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Chicken - Soup Base
1,200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Wanton Wrap
2,264,Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,"Veal - Inside, Choice","Fish - Scallops, Cold Smoked"
3,356,"Veal - Inside, Choice",Lamb - Ground,Pomello,Butter - Unsalted,"Tart Shells - Sweet, 4"
4,412,Olive - Spread Tapenade,Sprouts - Baby Pea Tendrils,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Soup - Campbells Bean Medley
...,...,...,...,...,...,...
995,97928,"Soup - Campbells, Lentil","Cheese - Brie,danish",Wiberg Super Cure,Pants Custom Dry Clean,Shrimp - 31/40
996,98069,Skirt - 29 Foot,Beans - Kidney White,Milk - 1%,Cheese - Taleggio D.o.p.,Sprouts - Baby Pea Tendrils
997,98159,"Lamb - Whole, Fresh",Bread - Raisin Walnut Oval,Chips Potato Salt Vinegar 43g,Tea - English Breakfast,Wine - Redchard Merritt
998,98185,Crackers - Trio,Pernod,Tea - Jasmin Green,Pastry - Choclate Baked,"Peas - Pigeon, Dry"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [82]:
distance_metric = 'correlation'

customer_similarity = pdist(pivot_table.T, metric=distance_metric)
customer_similarity_matrix = pd.DataFrame(squareform(customer_similarity), index=pivot_table.columns,\
                                          columns=pivot_table.columns)
customer_similarity_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,1.025211,0.887943,0.959400,0.972579,0.971542,1.051663,1.008305,1.003146,0.990052,...,1.026814,1.064510,1.026400,1.083910,1.000520,1.084359,1.027589,1.055680,1.035068,1.032223
200,1.025211,0.000000,1.001067,1.005715,1.073948,1.075028,0.926265,1.080738,1.003482,0.978399,...,0.981300,1.015439,0.930933,0.964814,1.030125,1.063924,0.870938,0.998990,1.064954,1.046097
264,0.887943,1.001067,0.000000,1.009730,0.947144,1.039546,0.987110,1.079281,0.991922,1.039807,...,0.965398,1.037471,1.001002,1.003357,0.931461,0.903754,0.953715,0.986098,0.938976,1.037179
356,0.959400,1.005715,1.009730,0.000000,1.040222,1.041743,0.992298,0.969268,1.071740,1.056114,...,1.003586,1.054821,1.005365,0.991127,1.023312,1.015781,1.065232,0.991223,0.974719,0.997480
412,0.972579,1.073948,0.947144,1.040222,0.000000,1.009738,0.979762,0.951334,1.024509,0.969367,...,1.048723,0.888568,1.032385,1.019126,0.981172,0.999745,1.077885,0.990375,0.976296,0.941690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,1.084359,1.063924,0.903754,1.015781,0.999745,0.968645,0.981220,1.002805,0.998928,1.029966,...,1.021932,0.981406,1.050846,0.960573,0.870999,0.000000,0.902651,0.964935,1.032316,1.070734
98069,1.027589,0.870938,0.953715,1.065232,1.077885,1.065183,1.012509,1.039542,1.018858,0.976137,...,1.043515,1.001031,1.027032,1.063468,0.930410,0.902651,0.000000,0.954242,0.995756,1.004420
98159,1.055680,0.998990,0.986098,0.991223,0.990375,1.003224,0.999040,0.991565,0.959505,0.991348,...,0.979906,0.913048,0.997792,1.044248,0.974952,0.964935,0.954242,0.000000,0.982682,1.017836
